# Exercise on Joins and anti-joins: add information from other tables

In [1]:
import pandas as pd

# Set some Pandas options: maximum number of rows/columns it's going to display
#pd.set_option('display.max_rows', 1000)
#pd.set_option('display.max_columns', 100)

# Load data from clinical trial

Data comes in two different files. The file `predimed_records.csv` file contains the clinical data for each patient, except which diet group they were assigned. The file `predimed_mapping.csv` contain the information of which patient was assigned to which diet group. 

In [2]:
df = pd.read_csv('../../data/predimed_records.csv')
df.head()

,patient-id,location-id,sex,age,bmi,waist,wth,htn,diab,hyperchol,famhist,hormo,p14,toevent,event
0,1,1,Male,58,33.53,122,0.753086,No,No,Yes,No,No,10,5.374401,Yes
1,1159,2,Male,77,31.05,119,0.730061,Yes,Yes,No,No,No,10,6.097194,No
2,509,3,Female,72,30.86,106,0.654321,No,Yes,No,Yes,No,8,5.946612,No
3,855,4,Male,71,27.68,118,0.694118,Yes,No,Yes,No,No,8,2.907598,Yes
4,511,3,Female,79,35.94,129,0.806250,Yes,No,Yes,No,No,9,4.761123,No


In [3]:
info = pd.read_csv('../../data/predimed_mapping.csv')
info.head()

,location-id,patient-id,group
0,2,301,MedDiet + Nuts
1,1,420,MedDiet + VOO
2,2,450,Control
3,5,57,MedDiet + VOO
4,2,411,Control


There were 5 different locations where the study was conducted, each one gave an identification number `patient-id` to each participant.

In [4]:
info['location-id'].unique()

array([2, 1, 5, 4, 3])

# 1. Add diet information to the patients' records

* For how many patients do we have clinical information? (i.e., rows in `df`)
* For how many patients do we have diet information? (i.e., rows in `info`)

In [5]:
len(df)

6324

In [6]:
len(info)

6287

Perform the merge, keeping in mind that it only make sense to analyze patients with the diet information. 
* Which type of merge would you do? 
* For how many patients do we have full information (records and which diet they followed? 

In [7]:
df_with_info = df.merge(info, on=['patient-id', 'location-id'], how='right')

In [8]:
df_with_info.count()

patient-id     6287
location-id    6287
sex            6287
age            6287
bmi            6287
waist          6287
wth            6287
htn            6287
diab           6287
hyperchol      6287
famhist        6287
hormo          5627
p14            6287
toevent        6287
event          6287
group          6287
dtype: int64

In [9]:
df_with_info

,patient-id,location-id,sex,age,bmi,waist,wth,htn,diab,hyperchol,famhist,hormo,p14,toevent,event,group
0,301,2,Female,75,31.07,110,0.654762,Yes,Yes,Yes,Yes,No,10,5.579740,No,MedDiet + Nuts
1,420,1,Female,69,29.41,86,0.623188,Yes,No,No,Yes,No,7,4.824093,No,MedDiet + VOO
2,450,2,Male,67,26.40,97,0.577381,Yes,Yes,Yes,No,No,4,2.012320,No,Control
3,57,5,Female,67,31.60,102,0.671053,Yes,Yes,Yes,Yes,No,6,2.532512,No,MedDiet + VOO
4,411,2,Female,68,27.24,91,0.590909,Yes,No,Yes,No,No,10,6.053388,No,Control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6282,697,5,Female,63,33.20,101,0.673333,Yes,Yes,Yes,No,No,5,5.987679,No,MedDiet + VOO
6283,943,2,Female,64,22.30,86,0.521212,Yes,Yes,No,No,No,8,5.442847,No,MedDiet + Nuts
6284,994,5,Female,68,32.89,102,0.662338,Yes,No,Yes,Yes,NaN,7,2.839151,No,MedDiet + Nuts
6285,210,5,Male,77,25.20,106,0.579235,No,Yes,Yes,No,No,10,2.496920,No,MedDiet + VOO


# 2. Remove drops from table

Some patients drop from the study early on and they should be removed from our analysis. Their IDS are stored in file `dropped.csv`.
1. Load the list of patients who droped, from `dropped.csv`
2. Use an anti-join to remove them from the table
3. How many patients (rows) are left in the data?

In [11]:
dropped = pd.read_csv('dropped.csv')

In [12]:
dropped.shape

(42, 2)

In [14]:
dropped.head()

,location-id,patient-id
0,3,587
1,4,899
2,3,807
3,5,1125
4,4,343


In [19]:
temp = df_with_info.merge(dropped, on=['location-id', 'patient-id'], how='outer', indicator=True)
temp

,patient-id,location-id,sex,age,bmi,waist,wth,htn,diab,hyperchol,famhist,hormo,p14,toevent,event,group,_merge
0,1,1,Male,58,33.53,122,0.753086,No,No,Yes,No,No,10,5.374401,Yes,Control,left_only
1,2,1,Female,80,28.15,77,0.527397,No,No,Yes,Yes,No,6,5.869952,No,MedDiet + VOO,left_only
2,3,1,Male,77,27.04,93,0.588608,Yes,No,No,Yes,No,11,4.720055,No,Control,left_only
3,4,1,Female,72,31.47,93,0.632653,No,Yes,No,No,No,10,6.151951,No,MedDiet + VOO,left_only
4,5,1,Male,68,27.58,101,0.594118,Yes,Yes,No,No,No,8,6.094456,No,MedDiet + Nuts,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6282,1299,5,Male,68,35.86,122,0.730539,Yes,No,Yes,Yes,No,10,2.956879,No,MedDiet + Nuts,left_only
6283,1300,5,Male,60,33.36,120,0.689655,Yes,Yes,No,Yes,No,8,2.346338,No,MedDiet + Nuts,left_only
6284,1301,5,Female,73,26.44,99,0.664430,Yes,Yes,Yes,Yes,No,7,2.954141,No,Control,left_only
6285,1302,5,Male,74,29.24,119,0.691860,No,Yes,No,No,No,12,5.382615,No,MedDiet + VOO,left_only


In [20]:
df_without_dropped = temp[temp['_merge'] == 'left_only'].drop('_merge', axis=1)

In [21]:
df_without_dropped.shape

(6245, 16)

In [22]:
df_without_dropped.head()

,patient-id,location-id,sex,age,bmi,waist,wth,htn,diab,hyperchol,famhist,hormo,p14,toevent,event,group
0,1,1,Male,58,33.53,122,0.753086,No,No,Yes,No,No,10,5.374401,Yes,Control
1,2,1,Female,80,28.15,77,0.527397,No,No,Yes,Yes,No,6,5.869952,No,MedDiet + VOO
2,3,1,Male,77,27.04,93,0.588608,Yes,No,No,Yes,No,11,4.720055,No,Control
3,4,1,Female,72,31.47,93,0.632653,No,Yes,No,No,No,10,6.151951,No,MedDiet + VOO
4,5,1,Male,68,27.58,101,0.594118,Yes,Yes,No,No,No,8,6.094456,No,MedDiet + Nuts


# 3. Save final result in `processed_data_predimed.csv`

1. Using the `.to_csv` method of Pandas DataFrames

In [23]:
df_without_dropped.to_csv('processed_data_predimed.csv', index=None)